In [81]:
from dotenv import load_dotenv
from pathlib import Path
import os

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import requests
from bs4 import BeautifulSoup

import random

import pandas as pd

import datetime

In [82]:
env_path = Path('...') / ".env"

load_dotenv(env_path)
client_id: str = os.getenv('CLIENT_ID')
client_secret: str = os.getenv('CLIENT_SECRET')

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [83]:
sp.album_tracks(album_id='0bLXUfNT34mna9aXq8ex68')['items'][0]['name']

'BLOOD.'

In [84]:
album = sp.album_tracks(album_id='79ONNoS4M9tfIA1mYLBYVX')['items']

track_names = []
for track in album:
    track_names.append(track['name'])

print(track_names)

['United In Grief', 'N95', 'Worldwide Steppers', 'Die Hard', 'Father Time (feat. Sampha)', 'Rich - Interlude', 'Rich Spirit', 'We Cry Together', 'Purple Hearts', 'Count Me Out', 'Crown', 'Silent Hill', 'Savior - Interlude', 'Savior', 'Auntie Diaries', 'Mr. Morale', 'Mother I Sober (feat. Beth Gibbons of Portishead)', 'Mirror', 'The Heart Part 5']


In [85]:
#scrape kworb
kworb_link = 'https://kworb.net/spotify/artist/2YZyLoL8N0Wb9xBt1NhZWg_songs.html'

headers = {
            'User-Agent': random.choice([
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
                'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)',
                'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0)',
            ])
        }

html_text = requests.get(kworb_link, headers=headers).text
soup = BeautifulSoup(html_text, 'lxml')

In [98]:
tr_elements = soup.find_all('tr')
data = []
for tr in tr_elements:
    td_text = tr.find('td', class_='text')
    if td_text:
        a_tag = td_text.find('a')
        if a_tag:
            track = a_tag.text
            # sp_link = a_tag['href']
            td_values = tr.find_all('td')
            streams = int(td_values[1].text.replace(',',''))
            data.append({'track_name': track,
                         "streams": streams,
                         "date" : datetime.datetime.now().strftime('%Y-%m-%d %I:%M %p')
                         })

In [99]:
df = pd.DataFrame(data)
df["track_name"] = pd.Categorical(df["track_name"], categories=track_names, ordered=True)
df = df.sort_values("track_name").reset_index(drop=True)
filtered_df = df[df['track_name'].isin(track_names)]

In [100]:
filtered_df.reset_index().drop(columns = 'index')
#can separate the table. one table is track id, name, album, artist. other table is streams, date 
#use sqlite3 for db. create two tables

,track_name,streams,date
0,United In Grief,284493677,2024-11-22 04:40 PM
1,N95,485878945,2024-11-22 04:40 PM
2,Worldwide Steppers,79993501,2024-11-22 04:40 PM
3,Die Hard,353261370,2024-11-22 04:40 PM
4,Father Time (feat. Sampha),193602890,2024-11-22 04:40 PM
5,Rich - Interlude,52352464,2024-11-22 04:40 PM
6,Rich Spirit,296878311,2024-11-22 04:40 PM
7,We Cry Together,94428260,2024-11-22 04:40 PM
8,Purple Hearts,103641079,2024-11-22 04:40 PM
9,Count Me Out,250217637,2024-11-22 04:40 PM
